### Module 3: Deep Dive - Functions, OOPs, Modules, Errors, and Exceptions


#### Case Study–3

#### Domain – E-Commerce
Business challenge/requirement
GoodsKart—the largest eCommerce company in Indonesia with revenue of $2B+
acquired another eCommerce company FairDeal. FairDeal has its own IT system to
maintain records of customers, sales, etc. For ease of maintenance and cost savings
GoodsKart is integrating customer databases of both organizations hence customer
data of FairDeal has to be converted in GoodsKart Customer Format.

Key issues
GoodsKart customer data has more fields than in FairDeal customer data. Hence
FairDeal data needs to be split and stored in GoodsKart Customer Object-Oriented
Data Structure

Considerations
The system should convert the data at run time

Business benefits
GoodsKart can eventually sunset the IT systems of FairDeal and reduce IT cost by 20-
30%
Approach to Solve
You have to use the fundamentals of Python taught in module 2
1. Read FairDealCustomerData.csv
2. Name field contains full name – use a regular expression to separate title, first
name, last name
3. Store the data in Customer Class
4. Create Custom Exception – CustomerNotAllowedException
5. Pass a customer to function "createOrder" and throw
CustomerNotAllowedException in case of blacklisted value is 1

Enhancements for code
You can try these enhancements in code
1. Change function createOrder to take productname and product code as input
2. Create Class Order
Return object of type Order in case customer is eligible

In [9]:
import csv
import re

# -------------------------------
# Custom Exception
# -------------------------------
class CustomerNotAllowedException(Exception):
    """Raised when a customer is blacklisted and not allowed to place an order."""
    pass

# -------------------------------
# Customer Class
# -------------------------------
class Customer:
    def __init__(self, customer_id, full_name, age, blacklisted):
        self.customer_id = customer_id
        self.title, self.first_name, self.last_name = self.split_name(full_name)
        self.age = int(age)
        self.blacklisted = int(blacklisted)

    def split_name(self, full_name):
        """
        Use regex to split title, first name, and last name.
        Example: 'Mr. John Doe' → ('Mr.', 'John', 'Doe')
        """
        pattern = r"^(?P<title>\w+\.)?\s*(?P<first>\w+)\s+(?P<last>\w+)$"
        match = re.match(pattern, full_name.strip())
        if match:
            return match.group("title"), match.group("first"), match.group("last")
        else:
            return None, full_name.strip(), ""  # fallback if regex fails

    def __str__(self):
        return f"Customer[{self.customer_id}] {self.title or ''} {self.first_name} {self.last_name}, Age: {self.age}, Blacklisted: {self.blacklisted}"

# -------------------------------
# Order Class
# -------------------------------
class Order:
    def __init__(self, order_id, customer, product_name, product_code):
        self.order_id = order_id
        self.customer = customer
        self.product_name = product_name
        self.product_code = product_code

    def __str__(self):
        return f"Order[{self.order_id}] for {self.customer.first_name} {self.customer.last_name}: {self.product_name} ({self.product_code})"

# -------------------------------
# Function to create order
# -------------------------------
def createOrder(customer, product_name, product_code):
    if customer.blacklisted == 1:
        raise CustomerNotAllowedException(
            f"Customer {customer.first_name} {customer.last_name} is blacklisted!"
        )
    # Generate order_id dynamically (could be UUID, here simple concatenation)
    order_id = f"ORD-{customer.customer_id}-{product_code}"
    return Order(order_id, customer, product_name, product_code)

# -------------------------------
# Main Program
# -------------------------------
# The error is in the Customer class initialization from CSV data
# Let's fix the main function where the CSV is processed

def main():
    filename = "FairDealCustomerData.csv"

    customers = []
    with open(filename, 'r') as file:
        reader = csv.DictReader(file)
        # Normalize headers (strip spaces, lowercase)
        headers = [h.strip().lower() for h in reader.fieldnames]
        
        # Print headers to debug what columns actually exist
        print("CSV Headers:", headers)

        for row in reader:
            normalized = {k.strip().lower(): v for k, v in row.items()}
            
            # Check if 'name' exists, otherwise look for alternatives like 'full_name', 'customer_name', etc.
            # You might need to adjust this based on your actual CSV structure
            name_field = None
            for possible_name in ['name', 'full_name', 'customer_name', 'first_name']:
                if possible_name in normalized:
                    name_field = possible_name
                    break
            
            if not name_field:
                print(f"Warning: No name field found for customer {normalized}")
                continue  # Skip this customer or provide a default name
            
            customer = Customer(
                customer_id=normalized.get('customerid') or normalized.get('customer id'),
                full_name=normalized[name_field],  # Use the found name field
                age=normalized['age'],
                blacklisted=normalized['blacklisted']
            )
            customers.append(customer)

    # Demo: Try creating orders for first 5 customers
    for cust in customers[:5]:
        print(cust)
        try:
            order = createOrder(cust, "Laptop", "P1001")
            print(order)
        except CustomerNotAllowedException as e:
            print("❌", e)

if __name__ == "__main__":
    main()


CSV Headers: ['braund', 'mr. owen harris', '1']
